In [1]:
# Movie Recommender App (Live API Version)

import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import os
from dotenv import load_dotenv
from IPython.display import HTML, display

# Load TMDb API Key
load_dotenv()
api_key = os.getenv("TMDB_API_KEY")

In [2]:
# Helper: Make API GET requests
def tmdb_get(endpoint, params=None):
    base_url = "https://api.themoviedb.org/3"
    if params is None:
        params = {}
    params['api_key'] = api_key
    params['language'] = 'en-US'
    response = requests.get(f"{base_url}/{endpoint}", params=params)
    return response.json()

In [3]:
# Get Genre List
def get_genres():
    data = tmdb_get("genre/movie/list")
    genres = {
        genre['name']: genre['id'] 
        for genre in data['genres'] 
        if genre['name'] not in ['Documentary', 'TV Movie']
    }
    return genres

In [4]:
# Get Top Rated Movies (optionally by genre)
def get_top_movies_by_genre(genre_id=None):
    url = "discover/movie" if genre_id else "movie/top_rated"
    params = {
        'sort_by': 'vote_average.desc',
        'vote_count.gte': 5000,
        'page': 1,
    }
    if genre_id:
        params['with_genres'] = genre_id
    data = tmdb_get(url, params)
    movies = data['results']
    return pd.DataFrame(movies)[['title', 'vote_average', 'vote_count', 'release_date']]

In [5]:
# Search Movie by Title
def search_movie(title):
    data = tmdb_get("search/movie", {'query': title})
    return data['results'][:1]

In [6]:
# Recommend Similar Movies
def get_recommendations(movie_id):
    data = tmdb_get(f"movie/{movie_id}/recommendations")
    return pd.DataFrame(data['results'])[['title', 'vote_average', 'overview']]

In [7]:
# Search Multiple Movies and Return Info
def search_multiple_movies(titles):
    info = []
    for title in titles:
        results = search_movie(title.strip())
        if results:
            movie = results[0]
            info.append({
                'title': movie['title'],
                'vote_average': movie['vote_average'],
                'vote_count': movie['vote_count'],
                'popularity': movie['popularity']
            })
    return pd.DataFrame(info)

In [8]:
# Function to fetch movies for a genre and year range
def get_movies_by_year_range(genre_id, start_year, end_year):
    all_movies = []
    for year in range(start_year, end_year + 1):
        params = {
            'with_genres': genre_id,
            'primary_release_year': year,
            'page': 1,
        }
        data = tmdb_get("discover/movie", params)
        all_movies.extend(data['results'])
    return all_movies

In [9]:
# Function to plot the rating trend
def plot_rating_trend(genre_name, start_year, end_year):
    with output:
        clear_output()
        genre_id = genre_dropdown.value
        if not genre_id:
            print("Please select a genre first.")
            return
        
        # Fetch movies for the given genre and year range
        movies = get_movies_by_year_range(genre_id, start_year, end_year)
        
        if not movies:
            print(f"No movies found for the genre '{genre_name}' between {start_year} and {end_year}.")
            return
        
        # Process the data (group by year and calculate average rating)
        ratings_by_year = {}
        for movie in movies:
            year = int(movie['release_date'][:4])
            rating = movie['vote_average']
            if year not in ratings_by_year:
                ratings_by_year[year] = []
            ratings_by_year[year].append(rating)
        
        # Calculate average rating per year
        avg_ratings = {year: np.mean(ratings) for year, ratings in ratings_by_year.items()}
        
        # Plot the data
        plt.figure(figsize=(10, 6))
        plt.plot(list(avg_ratings.keys()), list(avg_ratings.values()), marker='o', linestyle='-', color='b')
        plt.title(f"Average Movie Ratings for Genre: {genre_name} ({start_year}-{end_year})")
        plt.xlabel("Year")
        plt.ylabel("Average Rating")
        plt.grid(True)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

In [10]:
# UI Widgets
genres = get_genres()
genre_dropdown = widgets.Dropdown(
    options=[("-- All Genres --", None)] + list(genres.items()),
    description="Genre:",
    layout=widgets.Layout(width='50%')
)

search_box = widgets.Text(
    placeholder='Type a movie title...',
    description='Search:',
    layout=widgets.Layout(width='50%')
)

search_button = widgets.Button(
    description="Get Info",
    button_style='success'
)

genre_button = widgets.Button(
    description="Show Top Movies",
    button_style='info'
)

titles_input = widgets.Text(
    placeholder='Enter up to 5 movies, separated by commas',
    description='Movies:',
    layout=widgets.Layout(width='100%')
)

metric_dropdown = widgets.Dropdown(
    options=[('Rating', 'vote_average'), ('Vote Count', 'vote_count'), ('Popularity', 'popularity')],
    description='Compare by:',
    layout=widgets.Layout(width='50%')
)

compare_button = widgets.Button(
    description="Compare Movies",
    button_style='primary'
)

start_year_input = widgets.IntText(
    value=2000,
    description="Start Year:",
    layout=widgets.Layout(width='50%')
)

end_year_input = widgets.IntText(
    value=2024,
    description="End Year:",
    layout=widgets.Layout(width='50%')
)

rating_trend_button = widgets.Button(
    description="Show Rating Trend",
    button_style='warning'
)

output = widgets.Output()


In [11]:
def on_search_clicked(b):
    with output:
        clear_output()
        # Capture the input
        query = search_box.value
        search_box.value = ""  # Clear the search box
        # Display the search input at the top of the output
        display(HTML(f"<h3>Showing results for: <strong>{query}</strong></h3>"))
        
        # Search for the movie
        results = search_movie(query)
        if not results:
            print("No movie found.")
            return
        movie = results[0]
        
        # Display movie info and poster
        movie_poster_url = f"https://image.tmdb.org/t/p/w500{movie['poster_path']}" if movie.get('poster_path') else None
        movie_html = f"""
        <h4>{movie['title']}</h4>
        <p><strong>Rating:</strong> {movie['vote_average']}</p>
        <p><strong>Release Date:</strong> {movie['release_date']}</p>
        <p>{movie['overview']}</p>
        """
        if movie_poster_url:
            movie_html += f'<img src="{movie_poster_url}" alt="{movie["title"]}" style="width:200px;"/>'
        display(HTML(movie_html))
        
        # Get recommendations
        recommendations = get_recommendations(movie['id'])
        if not recommendations.empty:
            display(HTML("<h4>Recommended Movies:</h4>"))
            for _, rec_movie in recommendations.iterrows():
                rec_movie_poster_url = f"https://image.tmdb.org/t/p/w500{rec_movie.get('poster_path')}" if rec_movie.get('poster_path') else None
                rec_html = f"""
                <p><strong>{rec_movie['title']}</strong> ({rec_movie['vote_average']})</p>
                <p>{rec_movie['overview']}</p>
                """
                if rec_movie_poster_url:
                    rec_html += f'<img src="{rec_movie_poster_url}" alt="{rec_movie["title"]}" style="width:100px;"/>'
                display(HTML(rec_html))

search_button.on_click(on_search_clicked)

In [12]:
def on_genre_clicked(b):
    with output:
        clear_output()
        genre_id = genre_dropdown.value
        genre_dropdown.value = None
    
        if genre_id:
            genre_name = [name for name, gid in genres.items() if gid == genre_id][0]
            display(HTML(f"<h3>Top Movies for Genre: <strong>{genre_name}</strong></h3>"))
        else:
            display(HTML("<h3>Top Rated Movies (All Genres)</h3>"))

        df = get_top_movies_by_genre(genre_id)
        display(df.head(10))

genre_button.on_click(on_genre_clicked)

In [13]:
def on_compare_clicked(b):
    with output:
        clear_output()
        titles = titles_input.value.split(',')
        if len(titles) == 0:
            print("Please enter at least one movie title.")
            return
        df = search_multiple_movies(titles[:5])
        if df.empty:
            print("No valid movies found.")
            return
        metric = metric_dropdown.value
        display(HTML(f"<h3>Comparison by <strong>{metric.replace('_', ' ').title()}</strong></h3>"))
        plt.figure(figsize=(10, 6))
        plt.bar(df['title'], df[metric].astype(float), color='skyblue')
        plt.ylabel(metric.replace('_', ' ').title())
        plt.xlabel("Movie Titles")
        plt.title("Movie Comparison")
        plt.xticks(rotation=45, ha='right')

        if metric == 'vote_average':
            plt.ylim(0, 10)
            plt.yticks([i for i in range(11)])
        
        plt.tight_layout()
        plt.show()

compare_button.on_click(on_compare_clicked)

In [14]:
def on_rating_trend_clicked(b):
    genre_id = genre_dropdown.value
    genre_name = [name for name, gid in genres.items() if gid == genre_id][0] if genre_id else "All Genres"
    start_year = start_year_input.value
    end_year = end_year_input.value    
    plot_rating_trend(genre_name, start_year, end_year)

rating_trend_button.on_click(on_rating_trend_clicked)

In [15]:
# Display the UI
display(widgets.VBox([
    widgets.HTML("<h2>🎬 Movie Recommender App</h2>"),
    search_box,
    search_button,
    widgets.HTML("<hr>"),
    genre_dropdown,
    genre_button,
    widgets.HTML("<hr>"),
    widgets.HTML("<h3>📊 Compare Movies</h3>"),
    titles_input,
    metric_dropdown,
    compare_button,
    widgets.HTML("<hr>"),
    widgets.HTML("<h3>📈 Rating Trend by Genre Over Time</h3>"),
    genre_dropdown,
    start_year_input,
    end_year_input,
    rating_trend_button,
    widgets.HTML("<hr>"),
    output
]))